# Improvements

Perfusion<br>
Growth rate, and Product, the ratio figure<br>
Data export<br>

# Import Python Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)

# MeasuredData Class Test

## Cell Line 1

In [15]:
from CCDPApy.MeasuredData.MeasuredData import MeasuredData

f1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
for file in f1:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

Tesinting MeasuredData Class for CL1_1.xlsx.
Complete run_time() method.
Complete culture_volume() method.
Tesinting MeasuredData Class for CL1_2.xlsx.
Complete run_time() method.
Complete culture_volume() method.
Tesinting MeasuredData Class for CL1_3.xlsx.
Complete run_time() method.
Complete culture_volume() method.


In [19]:
md.get_pre_data().head()

,run_time_(days),run_time_(hrs),volume_before_sampling_(mL),volume_after_sampling_(mL)
0,0.000000,0.000000,74.0,73.0
1,0.997222,23.933333,73.0,71.5
2,1.609028,38.616667,71.5,70.0
3,1.991667,47.800000,70.0,68.5
4,2.613194,62.716667,68.5,67.0


## Cell Line 2

In [ ]:
from CCDPApy.MeasuredData.MeasuredData import MeasuredData
f2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
for file in f2:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data()

## Cell Line 3

In [ ]:
f3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']

for file in f3:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data()

# BioProcess Class Test

## In-Process

In in-process, the cumulative concentration will be calculated, <br>
and then specific rate will be calculated using the two-point calculation.

### Cell Line 1

There are the measurements of the feed concentration <br>
There are NOT measuremeents of the concentraion after feeding

In [5]:
from CCDPApy import BioProcess

files = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
#spc = ['Glucose','Lactate','Glutamine','Asparagine','Aspartate']

feed_concentration = True
concentration_after_feed = False

cell_line_1 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file,
                             #spc_list=spc,
                             #new_spc_list=['new'],
                             )
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_1[input_file.split('.')[0]] = bio_process
    
cell_line_1.keys()
exp_list = ['CL1_1', 'CL1_2', 'CL1_3']
exp = exp_list[0]

Tesinting BioProcess Class for CL1_1.xlsx.
Complete In-Process.
Tesinting BioProcess Class for CL1_2.xlsx.
Complete In-Process.
Tesinting BioProcess Class for CL1_3.xlsx.
Complete In-Process.


Display Experiment Information

In [6]:
cell_line_1[exp].disp_data(exp_info=True)


************ Experiment Information ************
Cell Line:              Sample CL1
Experiment ID:          Sample CL1_1
Experimenter Name:      CL1_1
Initial Culture Volume: 74.0 (mL)
Separate Feed List:
['GLUTAMINE']
Species List:
['ALANINE', 'ARGININE', 'ASPARAGINE', 'ASPARTATE', 'CYSTINE', 'GLUCOSE', 'GLUTAMINE', 'GLUTAMATE', 'GLYCINE', 'HISTIDINE', 'ISOLEUCINE', 'LACTATE', 'LEUCINE', 'LYSINE', 'METHIONINE', 'NH3', 'PHENYLALANINE', 'PROLINE', 'SERINE', 'THREONINE', 'TRYPTOPHAN', 'TYROSINE', 'VALINE', 'ETHANOLAMINE']


Data about Cell

In [20]:
cell = cell_line_1[exp].get_species('cell')
# DataFrame
cell.get_in_process.head()
# Display
# cell.disp_in_process

,run_time_(hrs),viable_cell_conc_(10^6_cells/mL),dead_cell_conc_(10^6_cells/mL),total_cell_conc_(10^6_cells/mL),ivcc_(x10^6_cells_hr/mL),cumCell_(x10^6_cells),viability_(%),qCell_(1/hr),kd,mv
0,0.000000,0.246,0.000000,0.246000,0.000000,0.000,100.0,NaN,NaN,NaN
1,23.883333,0.542,0.005475,0.547475,9.410033,21.608,99.0,0.031456,0.000582,0.032038
2,38.550000,0.940,0.005674,0.945674,20.278033,50.065,99.4,0.036621,0.000018,0.036640
3,47.583333,1.430,0.011532,1.441532,30.982533,84.365,99.2,0.045775,0.000547,0.046322
4,62.650000,3.110,0.076475,3.186475,65.183867,199.445,97.6,0.049121,0.001899,0.051020


In [ ]:
cell.

Data about Oxygen

In [8]:
oxy = cell_line_1[exp].get_species('oxygen')
# DataFrame
oxy.get_in_process
# Display
# oxy.disp_in_process

,run_time_(hrs),cumOxygen_(mmol),sOUR_(mmol/10^9_cells/hr),qOxygen_(mmol/109cell/hr)
0,0.000000,0.0,NaN,NaN
1,23.883333,0.0,NaN,0.419000
2,38.550000,0.0,NaN,0.359819
3,47.583333,0.0,NaN,0.333631
4,62.650000,0.0,NaN,0.328495
5,71.750000,0.0,NaN,0.320000
6,95.533333,0.0,NaN,0.152000
7,121.100000,0.0,NaN,0.136000
8,143.833333,0.0,NaN,0.135114
9,167.400000,0.0,NaN,0.110480


Data about Product(IgG)

In [9]:
product = cell_line_1[exp].get_species('product')
# DataFrame
product.get_in_process
# Display
# product.disp_in_process

,run_time_(hrs),cumProduct_(mg),qProduct_(mg/10^9_cell/hr)
0,0.000000,0.000000,NaN
1,23.883333,0.079116,0.115173
2,38.550000,0.128232,0.063207
3,47.583333,0.194380,0.088278
4,62.650000,0.306660,0.047926
5,71.750000,0.583638,0.121224
6,95.533333,1.643073,0.096033
7,121.100000,3.910237,0.102711
8,143.833333,4.337747,0.018206
9,167.400000,7.571374,0.117909


Data about other metabolite

In [11]:
glc = cell_line_1[exp].get_species('glucose')
glc.get_in_process

,run_time_(hrs),cumGlucose_(mmol),qGlucose_(mmol/10^9_cell/hr)
0,0.000000,0.000000,NaN
1,23.883333,0.151536,0.220599
2,38.550000,0.255331,0.133573
3,47.583333,0.430951,0.234375
4,62.650000,0.711053,0.119559
5,71.750000,0.869542,0.069365
6,95.533333,1.333046,0.042014
7,121.100000,2.214421,0.039929
8,143.833333,3.013392,0.034025
9,167.400000,3.950865,0.034183


In-process data

In [14]:
bio_process.get_pre_process()

AttributeError: 'BioProcess' object has no attribute '_pre_process'

### Cell Line 2

In [ ]:
from CCDPApy import BioProcess

files = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']

feed_concentration = False
concentration_after_feed = True

cell_line_2 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file)
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_2[input_file.split('.')[0]] = bio_process

In [ ]:
cell = bio_process.get_species('cell')
cell.get_in_process

In [ ]:
igg = bio_process.get_species('product')
igg.get_in_process

In [ ]:
ala = bio_process.get_species('alanine')
ala.get_in_process

### Cell Line 3

In [ ]:
from CCDPApy import BioProcess

files = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']

feed_concentration = False
concentration_after_feed = False

cell_line_3 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file)
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_3[input_file.split('.')[0]] = bio_process

## Display Pre-Process Data

In [ ]:
#bio_process.disp_data(exp_info=False, process='prepro')

## Post-Process

### Polynomial Regression

Cell Line 1

In [ ]:
cell_line_1

In [ ]:
poly_file = 'polynomial_order_1.xlsx'

for name, cl in cell_line_1.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

In [ ]:
cell = cell_line_1['CL1_1'].get_species('cell')
cell.get_post_process

Cell Line 2

In [ ]:
cell_line_2

In [ ]:
poly_file = 'polynomial_order_2.xlsx'

for name, cl in cell_line_2.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

In [ ]:
ala = cell_line_2['CL2_1'].get_species('alanine')
ala.get_in_process_data

Cell Line 3

In [ ]:
cell_line_3

In [ ]:
poly_file = 'polynomial_order_3.xlsx'

for name, cl in cell_line_3.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

### Rolling Window Polynomial Regression

Cell Line 1

In [ ]:
for name, cl in cell_line_1.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

In [ ]:
cl1_1 = cell_line_1['CL1_1']
cl1_1

In [ ]:
cl1_1.get_post_process_data()

In [ ]:
cell = cl1_1.get_species('cell')
cell.get_post_process_logistic

In [ ]:
bio_process.get_post_process_data()

In [ ]:
cl1_1 = cell_line_1['CL1_1']
ala = cl1_1.get_species('alanine')
ala._sp_rate_rollpoly, ala._t_mid

Cell Line 2

In [ ]:
for name, cl in cell_line_2.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

Cell Line 3

In [ ]:
for name, cl in cell_line_3.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

# Interactive Plot

In [1]:
from CCDPApy import CellLine    # used to aggregate all cell lines and experiments data
from CCDPApy import bioprocess_pipeline
from CCDPApy.BioProcess.BioProcess import BioProcess

input_files_1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
input_files_2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
input_files_3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']
measurement_sheet = 'Measured Data'
feed_sheet = 'Separate Feed Info'
poly_file = 'polynomial_order_1.xlsx'

cell_line = CellLine()

# Cell Line 1
for input in input_files_1:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=True, concentration_after_feed=False)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)
# Cell Line 2
for input in input_files_2:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=False, concentration_after_feed=True)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)
# Cell Line 3
for input in input_files_3:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=False, concentration_after_feed=False)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)

cell_line.disp_cell_lines()

cell_line.interactive_plot(mode='inline')

Cell Line: Sample CL1
Experiment 1: Sample CL1_1
Experiment 2: Sample CL1_2
Experiment 3: Sample CL1_3
Cell Line: Sample CL2
Experiment 1: Sample CL2_1
Experiment 2: Sample CL2_2
Experiment 3: Sample CL2_3
Cell Line: Sample CL3
Experiment 1: Sample CL3_1
Experiment 2: Sample CL3_2
Experiment 3: Sample CL3_3
Dash is running on http://127.0.0.1:8050/



## Growth rate

In [2]:
import plotly.express as px
df = cell_line.get_plot_data()
cell_df = df[(df['species']=='cell')]
df1 = cell_df[(cell_df['profile']=='concentration') & (cell_df['kind']!='viability')]
df2 = cell_df[cell_df['kind']=='viability']

fig1 = px.line(df1, x='runTime', y='value', title='Cell concentration', markers=True,
              color='kind', line_dash='cellLine', symbol='runID')
fig1.update_layout(yaxis_title='(10^6 cells/mL)')

df2 = df2.rename(columns={'value': 'viability'})
fig2 = px.line(df2, x='runTime', y='viability', markers=True,
               color='cellLine', line_dash='cellLine', symbol='runID',
               color_discrete_sequence=px.colors.qualitative.Pastel1,)

fig2.update_traces(yaxis='y2')
for fig_data in fig2.data:
    fig1.add_trace(fig_data)

fig1.update_layout(legend_x=1.1, legend_y=1)
fig1.update_layout(yaxis2={'side': 'right', 'title': 'viability (%)', 'overlaying': "y",})
fig1

In [3]:
df = cell_df[cell_df['profile']=='cumulative']
df1 = df[df['method']=='twoPoint']
df2 = df[df['method']=='polynomial']
fig1 = px.line(df1[df1['kind']=='integral'], x='runTime', y='value', 
                  title='Cell Cumulative concentration',
                  color='cellLine', line_dash='runID', symbol='cellLine')

fig2 = px.line(df1[df1['kind']=='cumulative'], x='runTime', y='value',
               color='cellLine', line_dash='runID', symbol='cellLine',
               color_discrete_sequence=px.colors.qualitative.Pastel1,)
fig2.update_traces(yaxis='y2')

for fig_data in fig2.data:
    fig1.add_trace(fig_data)
fig1.update_layout(legend_x=1.1, legend_y=1)
fig1.update_layout(yaxis2={'side': 'right', 'title': 'cumulative', 'overlaying': "y",})
fig1

In [4]:
df = cell_df[(cell_df['profile']=='spRate') & (cell_df['kind']=='growthRate')]
fig = px.line(df, x='runTime', y='value', 
              title='Specific Growth Rate',
              color='cellLine', line_dash='runID', symbol='method')
fig